# Simulation and Grading in Python

Running the simulation and computing the final score allows us to gather analytics data, find bottlenecks and update the schedules correspondingly. For example, we can increase the green times of streets where a lot of cars had to wait. We can also do some hill-climing by randomly adjusting schedules and using the grading function to see whether we improved.

In this notebook, you find ready-to-use data structures and functions that you can copy into your own code base to grade and improve your solutions.

At the very end, we try out the grader on the output of [Sample Submission with green light duration of one](https://www.kaggle.com/huikang/sample-submission-with-green-light-duration-of-one), which yields a score of 4020533.

In [ ]:
import os
from collections import deque, namedtuple

INPUT_FILE_PATH = '/kaggle/input/hashcode-2021-oqr-extension/hashcode.in'
OUTPUT_FILE_PATH = '/kaggle/input/sample-submission-with-green-light-duration-of-one/' \
                   'submission.csv'

In [ ]:
from typing import NamedTuple
import itertools
import heapq
import functools


class PQ(object):
    def __init__(self):
        self.heap = []

    def push(self, item, priority):
        heapq.heappush(self.heap, (priority, item))
        
    def __len__(self):
        return len(self.heap)

    def pop(self):
        return heapq.heappop(self.heap)[1]
    
    def top(self):
        return self.heap[0][1]
    

@functools.total_ordering
class Car(object):
    def __init__(self, id_):
        self.id = id_
        self.current_street_idx = 0
        self.reached_at = -1
        self.score = 0
        self.streets = []
        
    def time_left(self):
        tl = 0
        if self.reached_at == -1:
            for street in self.streets[self.current_street_idx + 1:]:
                tl += 1 + street.length
        return tl
    
    def __hash__(self):
        return hash(self.id)
    
    def __eq__(self, other):
        return self.id == other.id
    
    def __le__(self, other):
        return self.id <= other.id

    
class Street(object):
    def __init__(self, id_):
        self.id = id_
        self.name = ''
        self.length = -1
        self.queue = PQ()
        self.use_count = 0
    
    def __hash__(self):
        return hash(self.id)
    
    def __eq__(self, other):
        return self.id == other.id


class Intersection(object):
    def __init__(self, id_):
        self.id = id_
        self.incoming_streets = set()
        self.outgoing_streets = set()
        
    def __hash__(self):
        return hash(self.id)
    
    def __eq__(self, other):
        return self.id == other.id

In [ ]:
def read_input(f):
    street_name2o = {}
    duration, num_intersections, num_streets, num_cars, bonus = (int(x) for x in next(f).strip().split())
    intersections = [Intersection(i) for i in range(num_intersections)]
    streets = [Street(i) for i in range(num_streets)]
    cars = [Car(i) for i in range(num_cars)]
    
    print(num_streets)
    
    for i in range(num_streets):
        bid, eid, name, length = next(f).strip().split()
        bid, eid, length = int(bid), int(eid), int(length)
        street = streets[i]
        street.name = name
        street.length = length
        street_name2o[street.name] = street
        
        intersections[eid].incoming_streets.add(street)
        intersections[bid].outgoing_streets.add(street)
    
    
    for i in range(num_cars):
        route = next(f).strip().split()
        for j, street_name in enumerate(route[1:]):
            street = street_name2o[street_name]
            cars[i].streets.append(street)
            street.use_count += 1
            if j == 0:
                street.queue.push(cars[i], 0) # time left to reach the end of street
                
    return duration, bonus, streets, intersections, cars

In [ ]:
duration, bonus, streets, intersections, cars = read_input(open(INPUT_FILE_PATH))
max = 0
for i in streets:
    if i.use_count > max:
        max = i.use_count
print(max)

In [ ]:
def dumb_strat(streets, intersections, duration):
    """
    We will remove all unused streets from schedules and open each street for 1 second.
    Streets get priority in the schedule by (the top most waiting car's time to destination, queue size)
    Note this is a dumb strategy because we are not simulating anything
    """
    schedules = []
    for intersection in intersections:
        priority = []
        for street in intersection.incoming_streets:
            if len(street.queue):
                priority.append((street.queue.top().time_left(), len(street.queue), street.id, street.name))
            elif street.use_count:
                priority.append((duration + 1, 0, street.id, street.name))
        if priority:
            priority.sort(reverse=True)
            schedules.append(
                (intersection.id, [(street_name, 1) for _, _, _, street_name in priority])
            )
                
    return schedules

def dump_output(schedules, f):
    print(len(schedules))
    for intersection_id, street_pairs in schedules:
        print(intersection_id)
        print(len(street_pairs))
        for street_name, time in street_pairs:
            print(street_name, time)

In [ ]:
schedules = dumb_strat(streets, intersections, duration)

In [ ]:
dump_output(schedules, open('submission.csv', 'w'))

In [ ]:
print(score)